# Softmax Regression

softmax模型可以用来给不同的对象分配概率。<br>
在训练更加复杂的模型时，最后一步也往往需要用softmax来分配概率。<br>

softmax回归分两步：
* 首先 对某个待分类对象属于某个类的“证据”相加求和
* 然后 将这个“证据”的和和转化为概率

例如计算一张图片是否属于某类。<br>
使用加权的方法来累积计算这张图是否属于某类的“证据”。<br>
如果图片的像素强有力的体现该图不属于某个类，则权重的数值为负数。<br>
相反，则权重的数值为正。<br>
还需要引入额外的“证据”，称之为偏置量(bias)<br>
因此对于给定的输入图片$x$是属于第$i$类的总体“证据”可以表示为：

$$
evidence_i = \sum_{i}W_{i,j}x_j + b_i
$$

其中$W_i$代表权重，$b_i$代表第$i$类的偏置量，$j$代表给定图片$x$的像素索引 用于求和<br>
然后利用softmax函数可以把这些“证据”转化成概率$y$

$$
y = softmax(evidence)
$$

给定一张图$x$，它对于每一个类别的吻合度可以被softmax函数转换成一个概率值<br>
softmax函数可以定义为：

$$
softmax(x) = normalize(exp(x))
$$

展开右边可以得到：

$$
softmax(x)_i = \frac{exp(x_i)}{\sum_{j}exp(x_j)}
$$

假设模型里的权值不可以是0或是负数，softmax会正则化这些权重值，<br>
是它们的总和等于1，以此构建一个有效率的概率分布。

$$
\left[
\begin{matrix}
y_1 \\
y_2 \\
y_3
\end{matrix}
\right] = softmax\left(
\begin{matrix}
W_{1,1}x_1 + W_{1,2}x_1 + W_{1,3}x_1 + b_1 \\
W_{2,1}x_2 + W_{2,2}x_2 + W_{2,3}x_2 + b_2 \\
W_{3,1}x_3 + W_{3,2}x_3 + W_{3,3}x_3 + b_3
\end{matrix}
\right)
$$

转换成用 矩阵乘法和向量相加来表示：

$$
\left[
\begin{matrix}
y_1 \\
y_2 \\
y_3
\end{matrix}
\right] = softmax\left(
\left[\begin{matrix}
W_{1,1}&W_{1,2}&W_{1,3} \\
W_{2,1}&W_{2,2}&W_{2,3} \\
W_{3,1}&W_{3,2}&W_{3,3}
\end{matrix}\right]
\cdot
\left[\begin{matrix}
x_1 \\
x_2 \\
x_3
\end{matrix}\right]
+
\left[\begin{matrix}
b_1 \\
b_2 \\
b_3
\end{matrix}\right]
\right)
$$

简化为：
$$
y = softmax(W_x+b)
$$

## 评估模型
为了训练模型，通常需要定义一个指标来评估模型的好坏。<br>
这个指标被称为成本(cost)或是损失(loss)，两者意思相同，然后尽量最小化这个指标。<br>
非常常见的成本函数是“交叉熵”(cross-entropy):

$$
H_{y'}(y) = -\sum_{i}y'_i\log(y_i)
$$

其中y是预测的概率分布。<br>
y‘是实际的分布。<br>

## 计算交叉熵
（重要）<br>
交叉熵不仅仅用来衡量单一的一对预测和真实值，<br>
而是所有图片的交叉熵的总和。<br>
对100个数据点的预测的表示比单一数据点的预测的表示能更好的描述模型的性能。

In [2]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
### 创建模型----------------------------------

# 创建一个占位符x来保存输入值
# None表示第一个维度可以是任何长度（输入图片的个数）
# 第二个维度784=28x28是每一个图片的展平后的向量
x = tf.placeholder(tf.float32, [None, 784])

# 创建一个占位符来保存权重值W和偏置量b
# Variable常用来保存参数，计算中可以被修改，初始值为0
# 0~9共10个类别
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# y是预测出来的分类结果
# x乘以W(相当于W_x)再加上b，然后输入到softmax里面
evidence = tf.matmul(x, W) + b
y = tf.nn.softmax(evidence)

### 计算交叉熵----------------------------------

# 创建一个占位符y用来保存输出的正确值（正确分类）
y_ = tf.placeholder(tf.float32, [None, 10])

# 计算交叉熵
cross_entropy = tf.reduce_mean(
    -tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# cross_entropy = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))

# 优化方法使用梯度下降算法（gradient descent algorithm）,学习速率为0.5，来最小化交叉熵
# 会自动使用反向传播法（backpropagation algorithm）
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

### 训练和评估模型----------------------------------

# 建立一个交互式的会话
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# 评估
pred = tf.argmax(y, 1)
true = tf.argmax(y_, 1)
correct_prediction = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 循环训练1000次
# 每次训练会随机抓取训练数据中100个数据点
start = time.time()
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i % 100 == 0:
        train_acc = accuracy.eval(feed_dict={x: batch_xs, y_: batch_ys})
        test_acc = accuracy.eval(feed_dict={
            x: mnist.test.images,
            y_: mnist.test.labels
        })
        print "step % 5d" % (i), "train_acc={:5f}".format(
            train_acc), "test_acc={:5f}".format(test_acc)
    if i == 5 or i == 999:
        train_pred_sm = y.eval(feed_dict={x: batch_xs, y_: batch_ys})
        np.set_printoptions(suppress=True)
        print "train_pred_sm:", train_pred_sm[:20]
        train_pred = pred.eval(feed_dict={x: batch_xs, y_: batch_ys})
        train_true = true.eval(feed_dict={x: batch_xs, y_: batch_ys})
        test_pred = pred.eval(feed_dict={
            x: mnist.test.images,
            y_: mnist.test.labels
        })
        test_true = true.eval(feed_dict={
            x: mnist.test.images,
            y_: mnist.test.labels
        })
        print "train_pred:", train_pred[:20]
        print "train_true:", train_true[:20]
        print "test_pred:", test_pred[:20]
        print "test_true:", test_true[:20]
end = time.time()
print "Process Time(s):{:.2f}".format(end - start)

### 输出结果----------------------------------
print(sess.run(
    accuracy, feed_dict={
        x: mnist.test.images,
        y_: mnist.test.labels
    }))

step     0 train_acc=0.490000 test_acc=0.358900
train_pred_sm: [[0.00965373 0.0960416  0.0409211  0.25209102 0.1199263  0.08318483
  0.01621814 0.14114763 0.2229003  0.01791538]
 [0.01413021 0.00908886 0.06840898 0.14124462 0.05146847 0.0304097
  0.01488193 0.01460279 0.6550828  0.00068165]
 [0.00888283 0.00910231 0.03259642 0.03077484 0.529277   0.05112693
  0.02561535 0.10416036 0.1873413  0.0211226 ]
 [0.0265422  0.05091213 0.07684216 0.06405663 0.2536274  0.12787005
  0.0683338  0.08343241 0.18248521 0.06589801]
 [0.44239187 0.0078685  0.05784415 0.08579477 0.00743473 0.09756152
  0.0027383  0.00804953 0.2895752  0.00074149]
 [0.03030679 0.02441807 0.22968593 0.03752362 0.20505533 0.05168987
  0.26339245 0.02078422 0.13028656 0.00685719]
 [0.01608763 0.01761162 0.01423749 0.5314707  0.0075208  0.05613701
  0.00893051 0.07776545 0.26921567 0.00102312]
 [0.0562476  0.03790736 0.02530104 0.35246438 0.01400623 0.11266165
  0.00742584 0.02001391 0.37173095 0.002241  ]
 [0.0033518  0.085